<a href="https://colab.research.google.com/github/aepanfilov/shov/blob/main/Shov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# загрузка отдельного архива с сайта
!wget http://f1065011.xsph.ru/shov/original/noshov.zip
!wget http://f1065011.xsph.ru/shov/original/bok.zip
!wget http://f1065011.xsph.ru/shov/original/centr.zip
!wget http://f1065011.xsph.ru/shov/original/shov.zip
!wget http://f1065011.xsph.ru/shov/noshov2.zip

--2025-04-01 20:51:45--  http://f1065011.xsph.ru/noshov.zip
Resolving f1065011.xsph.ru (f1065011.xsph.ru)... 141.8.192.151
Connecting to f1065011.xsph.ru (f1065011.xsph.ru)|141.8.192.151|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59257935 (57M) [application/zip]
Saving to: ‘noshov.zip’

noshov.zip          100%[===================>]  56.51M  5.00MB/s    in 9.8s    

2025-04-01 20:51:55 (5.75 MB/s) - ‘noshov.zip’ saved [59257935/59257935]



In [ ]:
from google.colab import drive # Модуль для работы с Google Disk
drive.mount('/content/drive')
# использовать '\' для экранировани/я 'пробела' в пути
fname="/content/drive/MyDrive/ColabNotebooks/shov.zip"

In [3]:
!ls drive/Shareddrives/